### Trabalho 2 - Control Flow Automaton
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Enunciado

Este programa implementa a multiplicação de dois inteiros $\,a,b\,$ , fornecidos como “input”,    e com precisão limitada a $\,n\,$ bits (fornecido como parâmetro do programa). Note-se que

 - Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”. 
 - Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um “output” com as operações indicadas
 - Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo


  1. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste autómato. Para isso identifique as variáveis do modelo, o estado inicial e a relação de transição.
  2. Verifique se$\;$ $\;\mathsf{P}\,\equiv\,(x*y + z = a*b)\;$ $\;$é um invariante deste comportamento.

### Implementação

Começamos por importar o módulo `pysmt.shortcuts` que oferece uma API simplificada que disponibiliza as funcionalidades para a utilização usual de um SMT solver.
Os tipos estão definidos no módulo `pysmt.typing` de onde temos que importar o tipo `INT`. Definição dos inputs de $a$, $b$ e $n$.

In [10]:
from pysmt.shortcuts import *
from pysmt.typing import INT

#a,b,n = 5,4,5 # 20 usa 5 bits

#a,b,n = 6,8,6 # 48 usa 6 bits

a,b,n = 7,9,6 # 63 (111111) 6 bits

A seguinte função cria a $i$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.


In [2]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),BVType(n))
    state['y'] = Symbol('y'+str(i),BVType(n))
    state['z'] = Symbol('z'+str(i),BVType(n))
    return state

Definição da função `init` que, dado um possível estado do programa (um dicionário de variáveis), devolva um predicado do pySMT que testa se esse estado é um possível estado inicial do programa.

In [3]:
def init(state):
    return And(Equals(state['pc'], Int(0)), Equals(state['x'], BV(a,n)), Equals(state['y'], BV(b,n)), Equals(state['z'], BV(0,n)))

### Transições

Definição da função `trans` que, dados dois possíveis estados do programa, devolva um predicado do pySMT que testa se é possível transitar do primeiro para o segundo.

$$
\begin{array}{c}
t0 \rightarrow (\mathit{pc} = 0 \wedge \mathit{pc}' = 1 \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
t1 \rightarrow (\mathit{pc} = 1 \wedge y \neq 0 \wedge y \mod 2 = 1 \wedge \mathit{pc}' = 2 \wedge x' = x \wedge y' = y \wedge z' = z) \\
\vee\\
t2 \rightarrow (\mathit{pc} = 2 \wedge \mathit{pc}' = 1 \wedge x' = x \wedge y' = y - 1 \wedge z' = x + z) \\
\vee\\
t3 \rightarrow (\mathit{pc} = 1 \wedge y \neq 0 \wedge y \mod 2 = 0 \wedge \mathit{pc}' = 3 \wedge x' = x \wedge y' = y \wedge z' = z) \wedge z' \ge z) \\
\vee\\
t4 \rightarrow (\mathit{pc} = 3 \wedge x' = 2 * x \wedge y' = y/2 \wedge \mathit{pc}' = 1 \wedge x' \gt x \wedge y' = y \wedge z' = z) \wedge z' \ge z) \\
\vee\\
t5 \rightarrow (\mathit{pc} = 1 \wedge \mathit{pc}' = 4 \wedge y = 0)\\
\vee\\
t6 \rightarrow (\mathit{pc} = 4 \wedge \mathit{pc}' = 4 \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
t7 \rightarrow (\mathit{pc} = 3 \wedge \mathit{pc}' = 5 \wedge x' = 2 * x \wedge x' \le x \wedge y' = y \wedge z' = z)\\
\vee\\
t8 \rightarrow (\mathit{pc} = 2 \wedge \mathit{pc}' = 5 \wedge z' \lt z \wedge y' = y \wedge x' = x)\\
\vee\\
t9 \rightarrow (\mathit{pc} = 5 \wedge \mathit{pc}' = 5)\\
\end{array}
$$

In [4]:
def trans(curr,prox):
    t0 = And(
        Equals(curr['pc'], Int(0)), 
        Equals(prox['pc'], Int(1)), # do 0 para o estado 1
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    t1 = And(
        Equals(curr['pc'], Int(1)), 
        NotEquals(curr['y'], BVZero(n)),
        Equals(curr['y'] % 2,BVOne(n)),
        Equals(prox['pc'], Int(2)), # do 1 para o estado 2
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    t2 = And(
        Equals(curr['pc'], Int(2)), 
        Equals(prox['pc'], Int(1)), # do 2 para o estado 1
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y'] - 1),
        Equals(prox['z'], BVAdd(curr['x'],curr['z']))
    )
    t3 = And(
        Equals(curr['pc'], Int(1)), 
        NotEquals(curr['y'], BVZero(n)),
        Equals(curr['y'] % 2,BVZero(n)),
        Equals(prox['pc'], Int(3)), # do 1 para o estado 3
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z']),
        BVUGE(prox['z'],curr['z'])
    )
    t4 = And(
        Equals(curr['pc'], Int(3)), 
        Equals(prox['pc'], Int(1)), # do 3 para o estado 1
        Equals(prox['x'], 2 * curr['x']),
        Equals(prox['y'], curr['y'] / 2),
        BVUGT(prox['x'],curr['x']),
        Equals(prox['z'], curr['z'])
    )
    # stop
    t5 = And(
        Equals(curr['pc'], Int(1)), 
        Equals(curr['y'], BVZero(n)),
        Equals(prox['pc'], Int(4)), # do 1 para o estado 4
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    # no estado do stop
    t6 = And(
        Equals(curr['pc'], Int(4)), 
        Equals(prox['pc'], Int(4)), 
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    
    # overflow
    t7 = And(  
        Equals(curr['pc'], Int(3)), 
        Equals(prox['pc'], Int(5)), # do 3 para o overflow
        Equals(prox['x'], 2 * curr['x']),
        BVULE(prox['x'],curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    
    # overflow
    t8 = And(  
        Equals(curr['pc'], Int(2)), 
        Equals(prox['pc'], Int(5)), # do 2 para o overflow
        Equals(prox['x'],curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z']),
        BVULT(prox['z'],curr['z'])
    )
    
    # fica no overflow
    t9 = And(  
        Equals(curr['pc'], Int(5)), 
        Equals(prox['pc'], Int(5)),
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    
    return Or(t0,t1,t2,t3,t4,t5,t6,t7,t8,t9)

### Geração (Sem Invariante)

Criação dos vários estados, iniciação do predicado e adição das transições entre os estados.

In [5]:
def gera_traco(declare,init,trans,k):

    with Solver(name="z3") as s:
        trace = [declare(i) for i in range(k)]
        
        # adicionar o estado inicial 
        s.add_assertion(init(trace[0]))
        
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
            
        if s.solve():
            for i in range(k):
                print("Passo ",i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("------------------------")
                
gera_traco(declare,init,trans,20)

Passo  0
pc = 0
x = 7_5
y = 9_5
z = 0_5
------------------------
Passo  1
pc = 1
x = 7_5
y = 9_5
z = 0_5
------------------------
Passo  2
pc = 2
x = 7_5
y = 9_5
z = 0_5
------------------------
Passo  3
pc = 1
x = 7_5
y = 8_5
z = 7_5
------------------------
Passo  4
pc = 3
x = 7_5
y = 8_5
z = 7_5
------------------------
Passo  5
pc = 1
x = 14_5
y = 4_5
z = 7_5
------------------------
Passo  6
pc = 3
x = 14_5
y = 4_5
z = 7_5
------------------------
Passo  7
pc = 1
x = 28_5
y = 2_5
z = 7_5
------------------------
Passo  8
pc = 3
x = 28_5
y = 2_5
z = 7_5
------------------------
Passo  9
pc = 5
x = 24_5
y = 2_5
z = 7_5
------------------------
Passo  10
pc = 5
x = 24_5
y = 2_5
z = 7_5
------------------------
Passo  11
pc = 5
x = 24_5
y = 2_5
z = 7_5
------------------------
Passo  12
pc = 5
x = 24_5
y = 2_5
z = 7_5
------------------------
Passo  13
pc = 5
x = 24_5
y = 2_5
z = 7_5
------------------------
Passo  14
pc = 5
x = 24_5
y = 2_5
z = 7_5
------------------------
Passo  15


### Invariante

Verificar se$\;$ $\;\mathsf{P}\,\equiv\,(x*y + z = a*b)\;$ $\;$ é um invariante deste comportamento.

In [6]:
def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        with Solver(name="z3") as s:
            # completar
            
            trace = [declare(i) for i in range(k)]
        
            # adicionar o estado inicial 
            s.add_assertion(init(trace[0]))

            for i in range(k-1):
                s.add_assertion(trans(trace[i], trace[i+1]))
                
            # adicionar a negaçao do invariante
            s.add_assertion(Not(And(inv(trace[i]) for i in range(k-1))))

            if s.solve():
                print("A propriedade nao e invariante")
                return
    print(f'O invariante mantém-se nos primeiros {k} passos')


def invariante(state):
    return Equals(BVToNatural(state['x']) * BVToNatural(state['y']) + BVToNatural(state['z']), Int(a * b))

bmc_always(declare,init,trans,invariante,20)

A propriedade nao e invariante
